## Ankara için bir kira tahmini uygulaması

In [94]:
import os
import numpy as np
import pandas as pd

In [95]:
#csv dosyasındaki verileri 
df = pd.read_csv('hepsiEmlakAnkaraKiralikDaireler.csv')

In [96]:
#sütunları inceleme
print(df.columns)               

Index(['img-link href', 'photo-count', 'list-view-price', 'list-view-date',
       'list-view-title', 'left', 'celly', 'celly 2', 'celly 3', 'celly 4',
       'list-view-location', 'img-wrp href', 'he-lazy-image src', 'wp-btn',
       'listing-card--owner-info__firm-name', 'img-wrp href 2',
       'he-lazy-image src 2', 'listing-card--owner-info__name',
       'he-lazy-image src 3', 'list-view-header'],
      dtype='object')


In [97]:
#adresleri yeni bi df yapalım
addressDf = pd.DataFrame(df['list-view-location'].str.split('/', expand=True))
addressDf.columns = ["İl","İlçe","Mahalle"]
#burada tüm il sütununda Ankara yazdığı için belirli bir ölçüt değildir, zaten programın amacı Ankara'daki iller olduğu
#için il sütununu siliyoruz
addressDf.drop(columns="İl", inplace=True)
addressDf


,İlçe,Mahalle
0,Çankaya,Fakülteler Mah.
1,Çankaya,Bahçelievler Mah.
2,Keçiören,Yeşiltepe Mah.
3,Mamak,Ege Mah.
4,Çankaya,Gaziosmanpaşa Mah.
...,...,...
10325,Çankaya,Mutlukent Mah.
10326,Altındağ,Aydınlıkevler Mah.
10327,Keçiören,Kavacık Subayevleri Mah.
10328,Yenimahalle,Emniyet Mah.


In [98]:
#gereksiz sütunları kaldırma
df.drop(['photo-count', 'list-view-date', 'left', 'list-view-location', 'img-wrp href', 'he-lazy-image src', 'wp-btn', 'listing-card--owner-info__firm-name', 'img-wrp href 2', 'he-lazy-image src 2', 'listing-card--owner-info__name', 'he-lazy-image src 3', 'list-view-header','list-view-title'], axis=1,inplace=True)

In [99]:
#adresleri ve orijinal dataframe'imizi birleştirelim
df = pd.concat([df,addressDf], axis = 1)
df.head()

,img-link href,list-view-price,celly,celly 2,celly 3,celly 4,İlçe,Mahalle
0,https://www.hepsiemlak.com/ankara-cankaya-faku...,25.000,3 +\n 1,120 m²,40\n Yaşında,3. Kat,Çankaya,Fakülteler Mah.
1,https://www.hepsiemlak.com/ankara-cankaya-bahc...,38.000,2 +\n 1,90 m²,30\n Yaşında,1. Kat,Çankaya,Bahçelievler Mah.
2,https://www.hepsiemlak.com/ankara-kecioren-yes...,13.750,3 +\n 1,130 m²,25\n Yaşında,Yüksek Giriş,Keçiören,Yeşiltepe Mah.
3,https://www.hepsiemlak.com/ankara-mamak-ege-ki...,26.500,3 +\n 1,130 m²,13\n Yaşında,5. Kat,Mamak,Ege Mah.
4,https://www.hepsiemlak.com/ankara-cankaya-gazi...,55.000,5 +\n 1,225 m²,35\n Yaşında,Bahçe Katı,Çankaya,Gaziosmanpaşa Mah.


In [100]:
df.rename(columns={
    "img-link href": "Link",
    "list-view-price": "Fiyat",
    "celly": "Oda Sayısı",
    "celly 2": "m^2",
    "celly 3": "Bina Yaşı",
    "celly 4": "Kat"
}, inplace=True)

In [101]:
print(df['Oda Sayısı'].unique())

['3 +\n                          1' '2 +\n                          1'
 '5 +\n                          1' '4 +\n                          1'
 '1 +\n                          1' '4 +\n                          4'
 '6 +\n                          1' 'Stüdyo'
 '3 +\n                          2' '9 +\n                          3'
 '5 +\n                          0' '5 +\n                          2'
 '4 +\n                          0' '4 +\n                          2'
 '6 +\n                          2' '8 +\n                          1'
 '3 +\n                          0' '7 +\n                          1'
 '3 +\n                          3' '2 +\n                          0'
 '11 +\n                          3' '3 +\n                          5'
 '4 +\n                          5' '7 +\n                          2'
 '6 +\n                          6' '2 +\n                          2'
 '2 +\n                          3' '10 +\n                          1'
 '1 +\n                       

In [102]:
df['Oda Sayısı'] = df['Oda Sayısı'].apply(lambda x: x.replace('Stüdyo', '1 + 0') )
df['Oda Sayısı'] = df['Oda Sayısı'].apply(lambda x: x.replace('\n', '') )

In [103]:
df['Oda'] = df['Oda Sayısı'].apply(lambda x: x.split('+')[0] ).astype(int)
df['Salon'] = df['Oda Sayısı'].apply(lambda x: x.split('+')[1] ).astype(int)

In [104]:
df.drop(columns='Oda Sayısı', inplace=True)

In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10330 entries, 0 to 10329
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Link       10330 non-null  object
 1   Fiyat      10330 non-null  object
 2   m^2        10330 non-null  object
 3   Bina Yaşı  10326 non-null  object
 4   Kat        10127 non-null  object
 5   İlçe       10330 non-null  object
 6   Mahalle    10330 non-null  object
 7   Oda        10330 non-null  int64 
 8   Salon      10330 non-null  int64 
dtypes: int64(2), object(7)
memory usage: 726.5+ KB


In [106]:
df['m^2'].unique()

array(['120 m²', '90 m²', '130 m²', '225 m²', '110 m²', '220 m²',
       '200 m²', '105 m²', '160 m²', '150 m²', '135 m²', '145 m²',
       '65 m²', '85 m²', '118 m²', '75 m²', '125 m²', '40 m²', '95 m²',
       '155 m²', '115 m²', '140 m²', '45 m²', '180 m²', '175 m²',
       '100 m²', '350 m²', '43 m²', '168 m²', '210 m²', '165 m²',
       '190 m²', '1.450 m²', '170 m²', '195 m²', '138 m²', '285 m²',
       '330 m²', '640 m²', '70 m²', '55 m²', '108 m²', '3.540 m²',
       '60 m²', '700 m²', '35 m²', '162 m²', '62 m²', '260 m²', '300 m²',
       '83 m²', '230 m²', '80 m²', '128 m²', '450 m²', '127 m²', '50 m²',
       '139 m²', '54 m²', '185 m²', '240 m²', '98 m²', '107 m²', '313 m²',
       '151 m²', '412 m²', '250 m²', '164 m²', '30 m²', '96 m²', '154 m²',
       '159 m²', '82 m²', '183 m²', '58 m²', '72 m²', '67 m²', '117 m²',
       '158 m²', '1.105 m²', '86 m²', '276 m²', '360 m²', '149 m²',
       '94 m²', '124 m²', '280 m²', '99 m²', '1.120 m²', '126 m²',
       '122 m²', '375

In [107]:
df['m^2'] = df['m^2'].apply(lambda x: x.replace('m²', ''))
df['m^2'] = df['m^2'].apply(lambda x: x.replace('.', '')).astype(int)


In [108]:
df['m^2'].unique()

array([ 120,   90,  130,  225,  110,  220,  200,  105,  160,  150,  135,
        145,   65,   85,  118,   75,  125,   40,   95,  155,  115,  140,
         45,  180,  175,  100,  350,   43,  168,  210,  165,  190, 1450,
        170,  195,  138,  285,  330,  640,   70,   55,  108, 3540,   60,
        700,   35,  162,   62,  260,  300,   83,  230,   80,  128,  450,
        127,   50,  139,   54,  185,  240,   98,  107,  313,  151,  412,
        250,  164,   30,   96,  154,  159,   82,  183,   58,   72,   67,
        117,  158, 1105,   86,  276,  360,  149,   94,  124,  280,   99,
       1120,  126,  122,  375,  112,  132,  292,  102,  750,  123,  650,
        500,  430,  900,   97,  270,  167,  103,  113,   92,   89,   93,
         57,  161,  290,  232,  144,   78,  114,   66,  104, 1200,  142,
        137,  470,  116,  141,  255,  146,  235,  166,  400,   79,  310,
        242,   26,   52,  133,  197,   56,  216,  119,  101,  169,  245,
        163,  380,  184,  157,  399,  129,  201,  1

In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10330 entries, 0 to 10329
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Link       10330 non-null  object
 1   Fiyat      10330 non-null  object
 2   m^2        10330 non-null  int64 
 3   Bina Yaşı  10326 non-null  object
 4   Kat        10127 non-null  object
 5   İlçe       10330 non-null  object
 6   Mahalle    10330 non-null  object
 7   Oda        10330 non-null  int64 
 8   Salon      10330 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 726.5+ KB


In [110]:
df['Bina Yaşı'].unique()

array(['40\n                        Yaşında',
       '30\n                        Yaşında',
       '25\n                        Yaşında',
       '13\n                        Yaşında',
       '35\n                        Yaşında',
       '5\n                        Yaşında',
       '10\n                        Yaşında',
       '20\n                        Yaşında',
       '15\n                        Yaşında', 'Sıfır Bina',
       '3\n                        Yaşında',
       '50\n                        Yaşında',
       '7\n                        Yaşında',
       '28\n                        Yaşında',
       '29\n                        Yaşında',
       '17\n                        Yaşında',
       '36\n                        Yaşında',
       '45\n                        Yaşında',
       '18\n                        Yaşında',
       '22\n                        Yaşında',
       '43\n                        Yaşında',
       '4\n                        Yaşında',
       '38\n            

In [127]:
print(df['Bina Yaşı'].value_counts())  # Sütundaki değerlerin sıklığını göster
print(df['Bina Yaşı'].isnull().sum())  # NULL değer sayısını göster
print((df['Bina Yaşı'] == '').sum())   # Boş string ('') olan değerlerin sayısını göster
df['Bina Yaşı'] = df['Bina Yaşı'].replace('', '0')  # Boş stringleri sıfırla değiştir
df['Bina Yaşı'] = df['Bina Yaşı'].fillna('0')      # NULL değerleri sıfırla değiştir


Bina Yaşı
0      961
30     862
25     792
20     642
15     609
      ... 
167      1
58       1
123      1
54       1
220      1
Name: count, Length: 70, dtype: int64
0
4


In [132]:
df['Bina Yaşı'] = df['Bina Yaşı'].astype(str)  # Stringe dönüştür
df['Bina Yaşı'] = df['Bina Yaşı'].str.strip()  # Boşlukları temizle
df['Bina Yaşı'] = df['Bina Yaşı'].str.replace(r'\D', '', regex=True)  # Harf ve sembolleri temizle
df['Bina Yaşı'] = df['Bina Yaşı'].astype(int)  # Integer'e dönüştür



In [133]:
print(df['Bina Yaşı'].unique())


[ 40  30  25  13  35   5  10  20  15   0   3  50   7  28  29  17  36  45
  18  22  43   4  38  27  16   8  14  26   1   6  21  12  19  33  24  11
  42   2   9  31  23  32  54  60  55  39  47  34  51  37  44  48  41  49
  52  46 123  58 167 445 125 340 110  59  81  80 115 101 220]


In [134]:
kat_replace = {
    'Kot 1':'-1. Kat',
    'Kot 2':'-2. Kat',
    'Kot 3':'-3. Kat',
    'Yüksek Giriş':'1. Kat',
    'Ara Kat':'3. Kat',
    'En Üst Kat':'6. Kat',
    'Bahçe Katı': '0. Kat',
    'Giriş Katı' :'0. Kat',
    'Zemin':'0. Kat',
    '21 ve üzeri': '21. Kat',
    'Teras Katı': '6. Kat',
    'Çatı Katı' :'6. Kat',
    'Yarı Bodrum' : '0. Kat',
    'Bodrum': '0. Kat',
    'Bodrum ve Zemin':'0. Kat',
    'Villa Katı': '0. Kat'
}
df['Kat'] = df['Kat'].replace(kat_replace.keys(), kat_replace.values())
print(df['Kat'].unique())


['3. Kat' '1. Kat' '5. Kat' '0. Kat' '9. Kat' '2. Kat' '-1. Kat' '4. Kat'
 '14. Kat' '7. Kat' '-2. Kat' '6. Kat' nan '21. Kat' '10. Kat' '12. Kat'
 '8. Kat' '11. Kat' '18. Kat' '-3. Kat' '13. Kat' '16. Kat' '20. Kat'
 '19. Kat' '15. Kat' '17. Kat']


In [135]:
df.dropna(subset=['Kat'], inplace=True)
df['Kat'] = df['Kat'].apply(lambda x: x.replace('. Kat', ' '))
df['Kat'] = df['Kat'].apply(lambda x: x.replace(' ', '')).astype(int)
print(df['Kat'].unique())


[ 3  1  5  0  9  2 -1  4 14  7 -2  6 21 10 12  8 11 18 -3 13 16 20 19 15
 17]


In [136]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10127 entries, 0 to 10329
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Link       10127 non-null  object
 1   Fiyat      10127 non-null  object
 2   m^2        10127 non-null  int64 
 3   Bina Yaşı  10127 non-null  int64 
 4   Kat        10127 non-null  int64 
 5   İlçe       10127 non-null  object
 6   Mahalle    10127 non-null  object
 7   Oda        10127 non-null  int64 
 8   Salon      10127 non-null  int64 
dtypes: int64(5), object(4)
memory usage: 791.2+ KB


In [137]:
print(df['Fiyat'].unique())

['25.000' '38.000' '13.750' '26.500' '55.000' '31.500' '35.000' '30.000'
 '36.000' '21.500' '27.500' '39.000' '32.000' '14.250' '28.000' '15.000'
 '23.000' '16.300' '13.500' '17.000' '34.000' '20.000' '29.500' '26.000'
 '27.000' '20.500' '29.000' '22.500' '23.900' '22.000' '45.000' '16.000'
 '33.000' '59.000' '43.000' '19.000' '18.000' '18.500' '24.000' '13.000'
 '23.500' '57.000' '32.500' '17.500' '60.000' '21.000' '15.750' '14.500'
 '17.600' '14.000' '24.500' '15.400' '58.000' '15.500' '38.500' '40.000'
 '12.500' '44.000' '72.000' '25.500' '31.000' '18.750' '10.000' '37.000'
 '49.000' '10.500' '19.500' '54.000' '2.900' '14.750' '11.750' '12.000'
 '10.900' '50.000' '18.400' '13.150' '200.000' '63.000' '25.900' '29.250'
 '64.000' '49.500' '25.750' '42.000' '41.000' '34.950' '2.150.000'
 '16.500' '17.250' '28.500' '11.000' '29.750' '37.500' '20.750' '25.850'
 '185.000' '19.900' '17.400' '54.500' '24.750' '7.500' '6.000' '39.500'
 '150.000' '11.950' '51.000' '19.250' '95.000' '75.000' '5

In [138]:
df['Fiyat'] = df['Fiyat'].astype(str).apply(lambda x: x.replace('.',''))
df['Fiyat']= df['Fiyat'].astype(int)
print(df['Fiyat'].unique())

[     25000      38000      13750      26500      55000      31500
      35000      30000      36000      21500      27500      39000
      32000      14250      28000      15000      23000      16300
      13500      17000      34000      20000      29500      26000
      27000      20500      29000      22500      23900      22000
      45000      16000      33000      59000      43000      19000
      18000      18500      24000      13000      23500      57000
      32500      17500      60000      21000      15750      14500
      17600      14000      24500      15400      58000      15500
      38500      40000      12500      44000      72000      25500
      31000      18750      10000      37000      49000      10500
      19500      54000       2900      14750      11750      12000
      10900      50000      18400      13150     200000      63000
      25900      29250      64000      49500      25750      42000
      41000      34950    2150000      16500      17250      2

In [139]:
df.head()

,Link,Fiyat,m^2,Bina Yaşı,Kat,İlçe,Mahalle,Oda,Salon
0,https://www.hepsiemlak.com/ankara-cankaya-faku...,25000,120,40,3,Çankaya,Fakülteler Mah.,3,1
1,https://www.hepsiemlak.com/ankara-cankaya-bahc...,38000,90,30,1,Çankaya,Bahçelievler Mah.,2,1
2,https://www.hepsiemlak.com/ankara-kecioren-yes...,13750,130,25,1,Keçiören,Yeşiltepe Mah.,3,1
3,https://www.hepsiemlak.com/ankara-mamak-ege-ki...,26500,130,13,5,Mamak,Ege Mah.,3,1
4,https://www.hepsiemlak.com/ankara-cankaya-gazi...,55000,225,35,0,Çankaya,Gaziosmanpaşa Mah.,5,1


In [140]:
df['Oda Sayısı'] = (df['Oda'] + df['Salon']).astype(int)

In [141]:
#gereksiz sütunları kaldırma
df.drop(['Oda', 'Salon', 'Link'], axis=1,inplace=True)

In [142]:
columns = [ 'İlçe', 'Mahalle', 'Oda Sayısı','m^2','Bina Yaşı', 'Kat',  'Fiyat']
df = df[columns]
# DataFrame'in kopyasını oluştur (link kolonunu silmeden önce bu şekilde bi kalsın istedim)
df_copy = df.copy()


In [143]:
df['Mahalle'] = df['Mahalle'].apply(lambda x: x.replace(' Mah.', ''))
df.head()

,İlçe,Mahalle,Oda Sayısı,m^2,Bina Yaşı,Kat,Fiyat
0,Çankaya,Fakülteler,4,120,40,3,25000
1,Çankaya,Bahçelievler,3,90,30,1,38000
2,Keçiören,Yeşiltepe,4,130,25,1,13750
3,Mamak,Ege,4,130,13,5,26500
4,Çankaya,Gaziosmanpaşa,6,225,35,0,55000


In [144]:
#temizlediğimiz verileri yeni bir csv dosyasına aktaralım
df.to_csv('data.csv',index=False)

modeli ilk kurduğumuzda salon sayısını önemsiz ve oda sayısının önem düzeyini negatif olarak algıladı. bu durumu düzenleyebilmek için oda sayısı adında yeni bir sayısal kolon oluşturup oda sayısı değerlerinin toplamını gireceğiz yukarıda bu işlemleri yapıp devam edelim


In [145]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10127 entries, 0 to 10329
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   İlçe        10127 non-null  object
 1   Mahalle     10127 non-null  object
 2   Oda Sayısı  10127 non-null  int64 
 3   m^2         10127 non-null  int64 
 4   Bina Yaşı   10127 non-null  int64 
 5   Kat         10127 non-null  int64 
 6   Fiyat       10127 non-null  int64 
dtypes: int64(5), object(2)
memory usage: 632.9+ KB
